# Lateral-torsional Rayleigh-Ritz equations

In [ ]:
import numpy as np
from sympy import *
from bikewheelcalc import BicycleWheel, Rim, Hub, ModeMatrix
from bikewheelcalc.continuum_analysis import calc_continuum_stiff
from IPython.display import display

init_printing()

In [ ]:
# setup symbols
s = symbols('s', real=true)

# deformation
u = Function('u')
v = Function('v')
w = Function('w')
phi = Function('phi')

# Applied load
P = symbols('P', real=True)

# rim properties
R, EI2, GJ, EIw, mu, y0, r0 = symbols('R EI_2 GJ EI_w mu y_0, r_0',
                                      real=true, finite=true,
                                      positive=true)

t = symbols('t', real=true)  # Average net radial pull per unit length

# spoke properties
k_uu, k_up, k_pp = symbols('k_uu k_{u\phi} k_{\phi\phi}', real=true)

## Curvatures and total potential

In [ ]:
# curvatures
n = symbols('n', real=True, integer=True)
un, pn = symbols('u_n phi_n', real=True)
u = un*cos(n*s/R)
phi = pn*cos(n*s/R)

# Curvatures
k_2 = u.diff(s, 2) + phi/R
k_3 = u.diff(s)/R - phi.diff(s)


U_rim = (EI2*k_2**2 + GJ*k_3**2 + EIw*(k_3.diff(s))**2)/2

U_spokes = (k_uu*u**2 + 2*k_up*u*phi + k_pp*phi**2)/2

V_rim = R*t*(u.diff(s)**2 + (r0**2)*(phi.diff(s) - u.diff(s)/R)**2 +
         y0*(2*u.diff(s)*phi.diff(s) - phi**2/R))/2

PI = (integrate(U_rim + U_spokes - V_rim, (s, 0, 2*pi*R)).simplify()) - P*un

sub_r2 = [(y0**2, 0), (r0**2, 0)]
sub_y0 = [(y0, 0)]
PI = PI.subs(sub_r2 + sub_y0)

## Rayleigh-Ritz equations

In [ ]:
n_pos = symbols('n', real=True, integer=True, positive=True)
rr_eqns = [Eq(PI.subs(n, 0).diff(un)), Eq(PI.subs(n, 0).diff(pn)),
           Eq(PI.subs(n, n_pos).diff(un)), Eq(PI.subs(n, n_pos).diff(pn))]

display(rr_eqns[0])
display(rr_eqns[1])
display(rr_eqns[2])
display(rr_eqns[3])

## Solution

In [ ]:
s0 = solve(rr_eqns[0:2], (un, pn))

s0

In [ ]:
sn = solve(rr_eqns[2:4], (un, pn))

sn

## Mode stiffnesses

In [ ]:
K0 = 1/s0[un]
K0

In [ ]:
Kn = 1/sn[un]
Kn

## Numerically check against formula given in thesis

In [ ]:
w_r = 0.080
h_r = 0.005
t_r = 0.001

w = BicycleWheel()
w.hub = Hub(diam1=0.05, width1=0.025, width2=0.025)
w.rim = Rim.box(radius=0.3,
                young_mod=69.0e9, shear_mod=26.0e9,
                w=w_r, h=h_r, t=t_r)

eR = w.rim.radius
eEI = w.rim.young_mod*w.rim.I22
eGJ = w.rim.shear_mod*w.rim.I11

N = 2
kb = np.pi*eEI/eR**3*(N**2-1)**2
kt = np.pi*eGJ/eR**3*(N**2-1)**2*N**2

w.lace_radial(n_spokes=36, diameter=2.0e-3, young_mod=200e9, offset=0.01)

kk = calc_continuum_stiff(w)
kuu = kk[0, 0]
kup = kk[0, 3]
kpp = kk[3, 3]

mm = ModeMatrix(w)

In [ ]:
print(mm.lat_mode_stiff(n=0)/ 1000.)

print(2*np.pi*eR*(kuu - eR**2*kup**2/(eEI+eR**2*kpp)) / 1000.)

print(K0.subs([(P, 1), (R, eR), (n_pos, N), (t, 0),
               (EI2, eEI), (GJ, eGJ), (EIw, 0),
               (k_uu, kuu), (k_up, kup), (k_pp, kpp)]).evalf() / 1000.)

In [ ]:
N = 1

print(mm.lat_mode_stiff(n=1)/ 1000.)

a = eEI/eR**3 + eGJ/eR**3

print((np.pi*eR*kuu +
       (a*(np.pi*kpp/eR + 2*np.pi*kup) - np.pi*kup**2)/(a + kpp/eR))/ 1000.)

print(Kn.subs([(P, 1), (R, eR), (n_pos, N), (t, 0),
               (EI2, eEI), (GJ, eGJ), (EIw, 0),
               (k_uu, kuu), (k_up, kup), (k_pp, kpp)]).evalf() / 1000.)

In [ ]:
N = 2

print(mm.lat_mode_stiff(n=N)/ 1000.)

print((np.pi*eR*kuu +
       (kb*kt + (np.pi*kpp/eR)*(kb*N**4+kt)+2*np.pi*kup*(kb*N**2+kt)-(N**2-1)**2*np.pi**2*kup**2)/
       (kb+kt+(N**2-1)**2*np.pi*kpp/eR))/ 1000.)

print(Kn.subs([(P, 1), (R, eR), (n_pos, N), (t, 0),
         (EI2, eEI), (GJ, eGJ), (EIw, 0),
         (k_uu, kuu), (k_up, kup), (k_pp, kpp)]).evalf() / 1000.)